In [30]:
#%pip install git+https://github.com/cair/tmu.git
#%pip install numpy==1.26.4

In [31]:
import numpy as np
import pickle
import optuna
import os

from tmu.models.autoencoder.autoencoder import TMAutoEncoder

In [32]:
# Create folder models if it does not exist
os.makedirs("models", exist_ok=True)

In [33]:
def load_train_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)
    return X


def load_test_dataset(farm, event_id):
    X = np.loadtxt(f"./data_test/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take a sample of 5000 rows
    X = X[:5000]

    return X


def load_test_labels(farm, event_id):
    y = np.loadtxt(f"./data_test/y_{farm}_{event_id}.txt", dtype=np.uint32)
    y = np.array(y).astype(np.uint32)

    # Take a sample of 5000 rows
    y = y[:5000]

    return y


def load_thresh_dataset(farm, event_id):
    X = np.loadtxt(f"./data_train/X_{farm}_{event_id}.txt", dtype=np.uint32)
    X = np.array(X).astype(np.uint32)

    # Take the first 5000 rows
    X = X[:5000]

    return X


In [34]:
def save_model(tm: TMAutoEncoder, filename: str):
    a, d = tm.X_train, tm.encoded_X_train

    tm.X_train = None
    tm.encoded_X_train = None

    with open(f"./models/{filename}", "wb") as f:
        pickle.dump(tm, f)

    tm.X_train = a
    tm.encoded_X_train = d

In [35]:
# TODO: Load X_train from a dataset

train_datasets = [34, 7, 53, 27, 19, 77, 83, 52, 21, 2, 23, 87, 74, 86, 82]
test_dataset = [83, 52]

# Load all datasets into one array
X_train = np.concatenate([load_train_dataset("B", i) for i in train_datasets])

X_test = np.concatenate([load_test_dataset("B", i) for i in test_dataset])
y_test = np.concatenate([load_test_labels("B", i) for i in test_dataset])

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (704722, 504)
X_test shape: (7737, 504)
y_test shape: (7737,)


In [36]:
def hamming_loss(pred, X_test):
    """
    Computes the Hamming loss between predicted and ground truth binary arrays.

    Parameters:
    - pred (numpy array): Binary predictions of shape (n_samples, n_bits).
    - X_test (numpy array): Ground truth binary values of shape (n_samples, n_bits).

    Returns:
    - float: Hamming loss (fraction of incorrect bits).
    """
    assert pred.shape == X_test.shape, "Shapes of pred and X_test must match"

    # Compute the number of differing bits
    incorrect_bits = np.sum(pred != X_test)

    # Total number of bits
    total_bits = np.prod(X_test.shape)

    # Hamming loss is the fraction of incorrect bits
    return incorrect_bits / total_bits

In [37]:
def run_evaluation(tm: TMAutoEncoder) -> float:
    pred = tm.predict(X_test)

    loss = [hamming_loss(X_test[i], pred[i]) for i in range(len(X_test))]

    print(f"Mean loss: {np.mean(loss)}, Median loss: {np.median(loss)}, Max loss: {np.max(loss)}")

    # Mean loss for 0s
    return np.mean(loss)


def objective(trial: optuna.Trial) -> float:
    number_of_features = X_train.shape[1]
    output_active = np.arange(number_of_features, dtype=np.uint32)

    number_of_clauses = trial.suggest_int("number_of_clauses", 20, 2000)
    T = trial.suggest_int("T", 20, 10000)
    s = trial.suggest_int("s", 1, 100)
    max_included_literals = trial.suggest_int("max_included_literals", 1, 3 * number_of_features)
    accumulation = trial.suggest_int("accumulation", 1, 10)
    feature_negation = trial.suggest_categorical("feature_negation", [True, False])
    output_balancing = trial.suggest_float("output_balancing", 0, 10)
    number_of_examples = trial.suggest_int("number_of_examples", 100, 100)

    tm = TMAutoEncoder(
        number_of_clauses=number_of_clauses,
        T=T,
        s=s,
        output_active=output_active,
        max_included_literals=max_included_literals,
        accumulation=accumulation,
        feature_negation=feature_negation,
        platform="CPU",  # TODO: Change to CUDA
        output_balancing=output_balancing,
    )

    print(f"Starting training for trial {trial.number}")

    for e in range(5):
        tm.fit(X_train, number_of_examples=number_of_examples)

    # Write the current best result to file "temp_best.txt"
    with open("temp_params.txt", "w") as f:
        f.write(f"Trial: {trial.number}\n")

    return run_evaluation(tm)


In [38]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1)

# Save the best params to file
best_params = study.best_params

print(f"Best params: {best_params}")

with open("best_params.txt", "w") as f:
    for key, value in best_params.items():
        f.write(f"{key}: {value}\n")

Starting training for trial 0
Mean loss: 0.41300563566945886, Median loss: 0.4087301587301587, Max loss: 0.6785714285714286
Best params: {'number_of_clauses': 1382, 'T': 5368, 's': 44, 'max_included_literals': 871, 'accumulation': 1, 'feature_negation': True, 'output_balancing': 4.331756938281006, 'number_of_examples': 100}


In [39]:
best_params = study.best_params

print(best_params)

{'number_of_clauses': 1382, 'T': 5368, 's': 44, 'max_included_literals': 871, 'accumulation': 1, 'feature_negation': True, 'output_balancing': 4.331756938281006, 'number_of_examples': 100}
